In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import plotly.graph_objects as go
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
import logging as log
import scipy
import colorlog
import logging
weight_score_global = 0.7
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
gbc_model = GradientBoostingClassifier(random_state = 0)
abc_model = AdaBoostClassifier(random_state = 0)
rfc_model = RandomForestClassifier(random_state = 0)

In [2]:
X_train = pd.read_csv('/Users/irfanularifa/Documents/TA_FIX/0. Dataset/X_train.csv')
y_train = pd.read_csv('/Users/irfanularifa/Documents/TA_FIX/0. Dataset/y_train.csv')

In [3]:
y_train = y_train['Label']
y_train

0      1
1      1
2      1
3      1
4      0
      ..
155    0
156    1
157    1
158    1
159    0
Name: Label, Length: 160, dtype: int64

In [4]:
X_train = X_train.drop(['pIC50'],axis=1)
X_train

ALogP    ALogp2      AMR       apol  naAromAtom  nAromBond  nAtom  \
0   -0.3299  0.108834  39.5003  65.228653        21.0       24.0   53.0   
1   -1.1845  1.403040  56.3007  67.606411        15.0       17.0   59.0   
2    0.0239  0.000571  40.3193  66.479653        21.0       24.0   56.0   
3   -0.4296  0.184556  42.6228  76.076204        22.0       24.0   63.0   
4   -0.6399  0.409472  59.7313  78.211790        18.0       19.0   66.0   
..      ...       ...      ...        ...         ...        ...    ...   
155 -1.0395  1.080560  40.0833  65.908653        21.0       24.0   54.0   
156 -0.7613  0.579578  43.3476  68.575825        21.0       24.0   58.0   
157 -0.9778  0.956093  65.8072  85.634169        21.0       23.0   73.0   
158 -0.4468  0.199630  33.7560  59.041481        21.0       24.0   47.0   
159  0.5391  0.290629  40.1189  70.697825        22.0       24.0   59.0   

     nHeavyAtom    nH    nC  ...      SRW9          MW     WTPT-1     WTPT-3  \
0          32.0  21.0  24.0  ...  6.842683  448.130218  66.523208  23.122212   
1          32.0  27.0  24.0  ...  6.842683  438.206719  66.523208  23.515914   
2          35.0  21.0  25.0  ...  6.842683  482.156575  72.159057  27.759960   
3          35.0  28.0  30.0  ...  0.000000  464.209993  72.577369  13.576254   
4          36.0  30.0  30.0  ...  0.000000  482.220557  74.417748  16.538268   
..          ...   ...   ...  ...       ...         ...        ...        ...   
155        33.0  21.0  25.0  ...  6.842683  439.164440  68.549963  23.077169   
156        33.0  25.0  25.0  ...  6.842683  443.195740  68.532315  23.618312   
157        40.0  33.0  32.0  ...  7.002156  539.242021  83.404336  23.314449   
158        30.0  17.0  22.0  ...  8.074338  420.098918  62.014043  22.964834   
159        34.0  25.0  27.0  ...  0.000000  455.184506  69.214631  19.644256   

        WTPT-4     WTPT-5   WPATH  WPOL  XLogP  Zagreb  
0     7.980207  12.598607  3660.0  50.0  4.455   174.0  
1     7.980207  12.992309  3660.0  50.0  3.761   174.0  
2     7.980217  12.622195  4707.0  56.0  5.473   192.0  
3     7.383352   6.192902  4488.0  56.0  9.798   186.0  
4     9.962622   6.575646  4746.0  59.0  8.900   192.0  
..         ...        ...     ...   ...    ...     ...  
155   7.980215  15.096953  4007.0  52.0  3.707   178.0  
156   7.947877  15.670436  4051.0  50.0  4.322   178.0  
157  13.567397   9.747052  6854.0  66.0  7.504   216.0  
158   7.836884  12.584556  3071.0  45.0  3.576   166.0  
159  10.347942   9.296314  4246.0  55.0  7.564   180.0  

[160 rows x 507 columns]

In [5]:
X_valid = X_train
y_valid = y_train

In [6]:
class BaseOptimizationAlgorithm(ABC):

    def __init__(self,
                 objective_function,
                 n_iteration: int = 1000,
                 timeout: int = None,
                 population_size=50,
                 minimize=True,
                 logger=None,
                 **kwargs):
        self.kwargs=kwargs
        self.objective_function = objective_function
        self.minimize = minimize
        self.population_size = population_size
        self.n_iteration = n_iteration
        self.timeout = timeout
        self.my_logger=logger

    @abstractmethod
    def fit(self):
        pass

    def sigmoid(self, x):
        return 1/(1+np.exp(-x))

    def _evaluate_fitness(self, model, x_train, y_train, x_valid, y_valid,particle_swarm_flag=0,dragon_fly_flag=0):
        scores = []
        for i, individual in enumerate(self.individuals):
            chosen_features = [index for index in range(
                x_train.shape[1]) if individual[index] == 1]
            x_train_copy = x_train.iloc[:, chosen_features]
            x_valid_copy = x_valid.iloc[:, chosen_features]

            feature_hash = '_*_'.join(
                sorted(self.feature_list[chosen_features]))
            if feature_hash in self.feature_score_hash.keys():
                score = self.feature_score_hash[feature_hash]
            else:
                score = self.objective_function(
                    model, x_train_copy, y_train, x_valid_copy, y_valid, **self.kwargs)
                if not(self.minimize):
                    score = -score
                weight_score = weight_score_global
                total_feat = X_train.shape[1]
                len_cf = len(chosen_features)
                score_cf = 1 - (len_cf / total_feat)
                weight_cf = 1 - weight_score
                final_score = (weight_score * score) + (weight_cf * score_cf)
                self.feature_score_hash[feature_hash] = final_score

            if score < self.best_score:
                self.best_score = score
                self.best_dim = individual
                self.best_score_dimension = individual
            if particle_swarm_flag:
                if score < self.current_best_scores[i]:
                    self.current_best_scores[i] = score
                    self.current_best_individual_score_dimensions[i] = individual
            if dragon_fly_flag:
                if score > self.worst_score:
                    self.worst_score = score
                    self.worst_dim = individual
            scores.append(score)
        return scores

    def iteration_objective_score_monitor(self, i):
        if self.minimize:
            self.best_results_per_iteration[i] = {'best_score': self.best_score,
                                                  'objective_score': np.array(self.fitness_scores).min(),
                                                  'selected_features': list(self.feature_list[
                                                      np.where(self.individuals[np.array(self.fitness_scores).argmin()])[0]])}
        else:
            self.best_results_per_iteration[i] = {'best_score': -self.best_score,
                                                  'objective_score': -np.array(self.fitness_scores).min(),
                                                  'selected_features': list(self.feature_list[
                                                      np.where(self.individuals[np.array(self.fitness_scores).argmin()])[0]])}

    def initialize_population(self, x):
        self.individuals = np.random.randint(
            0, 2, size=(self.population_size, x.shape[1]))

    def _check_params(self, model, x_train, y_train, x_valid, y_valid):
        if (self.n_iteration <= 0):
            raise ValueError(
                f"n_init should be > 0, got {self.n_iteration} instead.")

        if (self.population_size <= 0):
            raise ValueError(
                f"population_size should be > 0, got {self.population_size} instead.")

        if (not (callable(self.objective_function))):
            raise TypeError(f"objective_function should be a callable function that returns\
                            metric value, got {type(self.objective_function)} instead")

        if y_train is None:
            raise ValueError(
                f"requires y_train to be passed, but the target y is None.")

        if x_train is None:
            raise ValueError(
                f"requires X_train to be passed, but the target X_train is None.")

        if (type(x_train) != pd.core.frame.DataFrame):
            raise TypeError(f" X_train should be of type pandas.core.frame.DataFrame,\
                            got {type(x_train)} instead.")

        if (type(x_valid) != pd.core.frame.DataFrame):
            raise TypeError(f" X_valid should be of type pandas.core.frame.DataFrame,\
                            got {type(x_valid)} instead.")

        if x_train.shape[1] != x_valid.shape[1]:
            raise ValueError(f" X_train and X_valid should have same number of features,\
                             got { x_train.shape[1]},{x_valid.shape[1]} instead.")

        if x_valid is None:
            raise ValueError(
                f"requires X_valid to be passed, but the target X_train is None.")

        if y_valid is None:
            raise ValueError(
                f"requires X_valid to be passed, but the target y_valid is None.")

        return_val = self.objective_function(
            model, x_train, y_train, x_valid, y_valid, **self.kwargs)
        if (not (isinstance(return_val, (int, float)))):
            raise TypeError(
                f"objective_function should return int/float value , got {type(return_val)} instead.")

    def plot_history(self):
        """
        Plot objective score history
        """
        res = pd.DataFrame.from_dict(self.best_results_per_iteration).T
        res.reset_index(inplace=True)
        res.columns = ['iteration', 'best_score',
                       'objective_score', 'selected_features']
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=res['iteration'], y=res['objective_score'],
                                 mode='markers', name='objective_score'))
        fig.add_trace(go.Scatter(x=res['iteration'], y=res['best_score'],
                                 mode='lines+markers',
                                 name='best_score'))
        fig.update_xaxes(title_text='Iteration')
        fig.update_yaxes(title_text='objective_score')
        fig.update_layout(
            title="Optimization History Plot")
#         fig.show()
        return fig

    def _check_individuals(self):
        if (self.individuals.sum(axis=1) == 0).sum() > 0:
            log.warning(str((self.individuals.sum(axis=1) ==
                        0).sum())+' individuals went zero')
            self.individuals[self.individuals.sum(axis=1) == 0] = np.random.randint(0, 2,
                                                                                    (self.individuals[self.individuals.sum(axis=1) == 0].shape[0],
                                                                                     self.individuals[self.individuals.sum(axis=1) == 0].shape[1]))


    def _setup_logger(self):
        logger = logging.getLogger()

        if (logger.hasHandlers()):
            logger.handlers.clear()

        # Logging info level to stdout with colors
        terminal_handler = colorlog.StreamHandler()
        color_formatter = colorlog.ColoredFormatter(
            "%(green)s [ %(asctime)s ] %(reset)s%(message)s",
            datefmt=None,
            reset=True,
            log_colors={
                'DEBUG':    'cyan',
                'INFO':     'green',
                'WARNING':  'yellow',
                'ERROR':    'red',
                'CRITICAL': 'red,bg_white',
            },
            secondary_log_colors={},
            style='%'
        )
        terminal_handler.setLevel(logging.DEBUG)
        terminal_handler.setFormatter(color_formatter)

        # Add handlers to logger
        logger.addHandler(terminal_handler)

        return logger

    
    def verbose_results(self,verbose, i):
        if verbose:
            if i==0:
                if self.my_logger==None:
                    self.my_logger = self._setup_logger()

            fitness_scores = np.array(self.fitness_scores).min() if self.minimize else -np.array(self.fitness_scores).min()
            best_score = self.best_score if self.minimize else -self.best_score

            self.my_logger.warning(f"Finished iteration #{i} with objective value {fitness_scores}. Current best value is {best_score} ")

In [7]:
class GeneticOptimization(BaseOptimizationAlgorithm):
    def __init__(self,
                 objective_function,
                 n_iteration: int = 1000,
                 timeout: int = None,
                 population_size=20,
                 selective_pressure=2,
                 elitism=2,
                 mutation_rate=0.05,
                 minimize=False,
                 logger=None,
                 **kwargs):
        """
        Parameters
        ----------
        objective_function : user made function of the signature 'func(model,X_train,y_train,X_test,y_test)'
            The function must return a value, that needs to be minimized/maximized.
        n_iteration : int, default=1000
            Number of time the Optimization algorithm will run
        timeout: int = None
            Stop operation after the given number of second(s).
            If this argument is set to None, the operation is executed without time limitation and n_iteration is followed
        population_size : int, default=50
            Total size of the population
        selective_pressure : int, default=2
            measure of reproductive opportunities for each organism in the population
        elitism : int, default=2
            number of top individuals to be considered as elites
        mutation_rate :  float, default=0.05
            rate of mutation in the population's gene
        minimize : bool, default=True
            Defines if the objective value is to be maximized or minimized
        logger: Logger or None, optional (default=None)
            - accepts `logging.Logger` instance.
            
        **kwargs
            Any extra keyword argument for objective_function
        Attributes
        ----------
        best_feature_list : ndarray of shape (n_features)
            list of features with the best result of the entire run
        """
        super().__init__(objective_function, n_iteration, timeout, population_size, minimize, logger, **kwargs)
        self.n_generations = n_iteration
        self.selective_pressure = selective_pressure
        self.elitism = elitism
        self.mutation_rate = mutation_rate

    def _evaluate_fitness(self, model, x_train, y_train, x_valid, y_valid):
        scores = []
        for individual in self.individuals:
            chosen_features = [index for index in range(
                x_train.shape[1]) if individual[index] == 1]
            x_train_copy = x_train.iloc[:, chosen_features]
            x_valid_copy = x_valid.iloc[:, chosen_features]
            feature_hash = '_*_'.join(
                sorted(self.feature_list[chosen_features]))
            if feature_hash in self.feature_score_hash.keys():
                score = self.feature_score_hash[feature_hash]
            else:
                score = self.objective_function(
                    model, x_train_copy, y_train, x_valid_copy, y_valid, **self.kwargs)
                if self.minimize:
                    score = -score
                self.feature_score_hash[feature_hash] = score

            
            scores.append(score)

        self.fitness_scores = scores
        current_best_score = np.max(self.fitness_scores)
        if current_best_score > self.best_score:
            self.best_score = current_best_score
            self.best_feature_set = self.individuals[np.argmax(
                self.fitness_scores), :]

        ranks = scipy.stats.rankdata(scores, method='average')
        self.fitness_ranks = self.selective_pressure * ranks

    def _select_individuals(self, model, x_train, y_train, x_valid, y_valid):
        self._evaluate_fitness(model, x_train, y_train, x_valid, y_valid)

        sorted_individuals_fitness = sorted(
            zip(self.individuals, self.fitness_ranks), key=lambda x: x[1], reverse=True)
        elite_individuals = np.array(
            [individual for individual, fitness in sorted_individuals_fitness[:self.elitism]])

        non_elite_individuals = np.array(
            [individual[0] for individual in sorted_individuals_fitness[self.elitism:]])

        non_elite_individuals_fitness = [
            individual[1] for individual in sorted_individuals_fitness[self.elitism:]]
        selection_probability = non_elite_individuals_fitness / \
            np.sum(non_elite_individuals_fitness)

        selected_indices = np.random.choice(range(
            len(non_elite_individuals)), self.population_size//2, p=selection_probability)
        selected_individuals = non_elite_individuals[selected_indices, :]
        self.fit_individuals = np.vstack(
            (elite_individuals, selected_individuals))

    # Make me a mutant!
    def _mutate(self, array):
        mutated_array = np.copy(array)
        for idx, gene in enumerate(array):
            if np.random.random() < self.mutation_rate:
                array[idx] = 1 if gene == 0 else 0

        return mutated_array

    def _produce_next_generation(self):
        new_population = np.empty(
            shape=(self.population_size, self.individuals.shape[1]), dtype=np.int32)
        for i in range(0, self.population_size, 2):
            parents = self.fit_individuals[np.random.choice(
                self.fit_individuals.shape[0], 2, replace=False), :]
            crossover_index = np.random.randint(0, len(self.individuals[0]))
            new_population[i] = np.hstack(
                (parents[0][:crossover_index], parents[1][crossover_index:]))
            new_population[i+1] = np.hstack(
                (parents[1][:crossover_index], parents[0][crossover_index:]))

            new_population[i] = self._mutate(new_population[i])
            new_population[i+1] = self._mutate(new_population[i+1])
        self.individuals = new_population

    def _verbose_results(self, verbose, i):
        if verbose:
            if i == 0:
                print(
                    "\t\t Best value of metric across iteration \t Best value of metric across population  ")
            if self.minimize:
                print(
                    f"Iteration {i} \t {-np.array(self.fitness_scores).max()} \t\t\t\t\t {-self.best_score} ")
            else:
                print(
                    f"Iteration {i} \t {np.array(self.fitness_scores).max()} \t\t\t\t\t {self.best_score} ")

    def _iteration_objective_score_monitor(self, i):
        if self.minimize:
            self.best_results_per_iteration[i] = {'best_score': -self.best_score,
                                                  'objective_score': -np.array(self.fitness_scores).max(),
                                                  'selected_features': list(self.feature_list[
                                                      np.where(self.individuals[np.array(self.fitness_scores).argmin()])[0]])}
        else:
            self.best_results_per_iteration[i] = {'best_score': self.best_score,
                                                  'objective_score': np.array(self.fitness_scores).max(),
                                                  'selected_features': list(self.feature_list[
                                                      np.where(self.individuals[np.array(self.fitness_scores).argmin()])[0]])}

    def fit(self, model, X_train, y_train, X_valid, y_valid, verbose=True):
        """
        Parameters
        ----------      
        model : machine learning model's object
           machine learning model's object
        X_train : pandas.core.frame.DataFrame of shape (n_samples, n_features)
           Training input samples to be used for machine learning model
        y_train : pandas.core.frame.DataFrame or pandas.core.series.Series of shape (n_samples)
           The target values (class labels in classification, real numbers in regression).
        X_valid : pandas.core.frame.DataFrame of shape (n_samples, n_features)
           Validation input samples
        y_valid : pandas.core.frame.DataFrame or pandas.core.series.Series of shape (n_samples)
            The target values (class labels in classification, real numbers in regression).                
        verbose : bool,default=True
             Print results for iterations
        """
        self._check_params(model, X_train, y_train, X_valid, y_valid)

        self.feature_score_hash = {}
        self.feature_list = np.array(list(X_train.columns))
        self.best_results_per_iteration = {}
        self.best_score = np.inf
        self.best_dim = np.ones(X_train.shape[1])

        self.initialize_population(X_train)
        self.best_score = -1 * float(np.inf)
        self.best_scores = []

        if (self.timeout is not None):
            timeout_upper_limit = time.time() + self.timeout
        else:
            timeout_upper_limit = time.time()
        for i in range(self.n_generations):

            if (self.timeout is not None) & (time.time() > timeout_upper_limit):
                warnings.warn("Timeout occured")
                break
            self._select_individuals(model, X_train, y_train, X_valid, y_valid)
            self._produce_next_generation()
            self.best_scores.append(self.best_score)

            self._iteration_objective_score_monitor(i)
            self._verbose_results(verbose, i)
            self.best_feature_list = list(
                self.feature_list[np.where(self.best_dim)[0]])
        return self.best_feature_list

In [8]:
def objective_function_topass(model,X_train, y_train, X_valid, y_valid):      
    #  Cross Val Score  
    P = cross_val_score(model,X_train,y_train,cv=10, scoring="accuracy")
    P = np.average(P)
    return P

def feature_selection(model, X_train, y_train, X_valid, y_valid):
    algo_object=GeneticOptimization(objective_function_topass,n_iteration=50,
                                population_size=20,selective_pressure=2,elitism=2,
                                mutation_rate=0.05,minimize=False)
    algo_object.fit(model ,X_train, y_train, X_valid, y_valid, verbose=True)
    return algo_object

In [9]:
def multiple_runtime(n, model, X_train, y_train, X_valid, y_valid):
    run_idx = []
    num_features = []
    objective_scores = []
    best_scores = []
    features = []
    plots = []
    
    for i in range(n):
        print("\n\n ----------------------------------------------------------------------------------------")
        print(f"Run Number - {i+1}")
        best_score = 0
        
        solution = feature_selection(model, X_train, y_train, X_valid, y_valid)
        #  Best Solution Index
        for j in range(len(solution.best_results_per_iteration)):
            if solution.best_results_per_iteration[j]['objective_score'] > best_score:
                best_score = solution.best_results_per_iteration[j]['objective_score']
                index = j

        #  Solution per iteration
        num_feature = len(solution.best_results_per_iteration[index]['selected_features'])
        solution_objective_score = solution.best_results_per_iteration[index]['objective_score']
        solution_best_score = solution.best_results_per_iteration[index]['best_score']
        feature = solution.best_results_per_iteration[index]['selected_features']
        plot = solution.plot_history()
                       
                       
        run_idx.append(i+1)
        num_features.append(num_feature)
        objective_scores.append(solution_objective_score)
        best_scores.append(solution_best_score)
        features.append(feature)
        plots.append(plot)
                       
        col_name = {'run_idx': run_idx, 'objective_scores' : objective_scores, 
                         'best_scores' : best_scores, 'num_features': num_features, 'selected_features' : features, 'plot' : plots}
        df_solution = pd.DataFrame(data=col_name)
    return df_solution

# Random Forest

In [11]:
start = time.time()

random_forest_df = multiple_runtime(20, rfc_model, X_train, y_train, X_valid, y_valid)

end = time.time()
print("time =" ,((end - start)%60), " Menit")
print("\n\n ----------------------------------------------------------------------------------------")
random_forest_df


# Ada Boost 

In [21]:
start = time.time()

ada_boost_df = multiple_runtime(20, abc_model, X_train, y_train, X_valid, y_valid)

end = time.time()
print("time =" ,((end - start)%60), " Menit")
print("\n\n ----------------------------------------------------------------------------------------")
ada_boost_df



 ----------------------------------------------------------------------------------------
Run Number - 1
		 Best value of metric across iteration 	 Best value of metric across population  
Iteration 0 	 0.7375 					 0.7375 
Iteration 1 	 0.74375 					 0.74375 
Iteration 2 	 0.75625 					 0.75625 
Iteration 3 	 0.7375 					 0.75625 
Iteration 4 	 0.7375 					 0.75625 
Iteration 5 	 0.74375 					 0.75625 
Iteration 6 	 0.74375 					 0.75625 
Iteration 7 	 0.74375 					 0.75625 
Iteration 8 	 0.74375 					 0.75625 
Iteration 9 	 0.74375 					 0.75625 
Iteration 10 	 0.74375 					 0.75625 
Iteration 11 	 0.74375 					 0.75625 
Iteration 12 	 0.74375 					 0.75625 
Iteration 13 	 0.74375 					 0.75625 
Iteration 14 	 0.74375 					 0.75625 
Iteration 15 	 0.74375 					 0.75625 
Iteration 16 	 0.74375 					 0.75625 
Iteration 17 	 0.74375 					 0.75625 
Iteration 18 	 0.74375 					 0.75625 
Iteration 19 	 0.74375 					 0.75625 
Iteration 20 	 0.74375 					 0.75625 
Iteration 21 	 0.74375 		

Iteration 1 	 0.74375 					 0.74375 
Iteration 2 	 0.75625 					 0.75625 
Iteration 3 	 0.75 					 0.75625 
Iteration 4 	 0.74375 					 0.75625 
Iteration 5 	 0.74375 					 0.75625 
Iteration 6 	 0.7625 					 0.7625 
Iteration 7 	 0.7625 					 0.7625 
Iteration 8 	 0.75625 					 0.7625 
Iteration 9 	 0.75625 					 0.7625 
Iteration 10 	 0.75625 					 0.7625 
Iteration 11 	 0.75 					 0.7625 
Iteration 12 	 0.74375 					 0.7625 
Iteration 13 	 0.75 					 0.7625 
Iteration 14 	 0.75 					 0.7625 
Iteration 15 	 0.75 					 0.7625 
Iteration 16 	 0.75 					 0.7625 
Iteration 17 	 0.75 					 0.7625 
Iteration 18 	 0.75 					 0.7625 
Iteration 19 	 0.75 					 0.7625 
Iteration 20 	 0.75 					 0.7625 
Iteration 21 	 0.75 					 0.7625 
Iteration 22 	 0.7625 					 0.7625 
Iteration 23 	 0.7625 					 0.7625 
Iteration 24 	 0.7625 					 0.7625 
Iteration 25 	 0.7625 					 0.7625 
Iteration 26 	 0.7625 					 0.7625 
Iteration 27 	 0.7625 					 0.7625 
Iteration 28 	 0.7625 					 0.7625 
Iteration 

Iteration 9 	 0.7625 					 0.7625 
Iteration 10 	 0.7625 					 0.7625 
Iteration 11 	 0.7625 					 0.7625 
Iteration 12 	 0.7625 					 0.7625 
Iteration 13 	 0.7625 					 0.7625 
Iteration 14 	 0.7625 					 0.7625 
Iteration 15 	 0.7625 					 0.7625 
Iteration 16 	 0.7625 					 0.7625 
Iteration 17 	 0.7625 					 0.7625 
Iteration 18 	 0.7625 					 0.7625 
Iteration 19 	 0.7625 					 0.7625 
Iteration 20 	 0.7625 					 0.7625 
Iteration 21 	 0.7625 					 0.7625 
Iteration 22 	 0.7625 					 0.7625 
Iteration 23 	 0.7625 					 0.7625 
Iteration 24 	 0.7625 					 0.7625 
Iteration 25 	 0.7625 					 0.7625 
Iteration 26 	 0.7625 					 0.7625 
Iteration 27 	 0.7625 					 0.7625 
Iteration 28 	 0.7625 					 0.7625 
Iteration 29 	 0.7625 					 0.7625 
Iteration 30 	 0.7625 					 0.7625 
Iteration 31 	 0.7625 					 0.7625 
Iteration 32 	 0.7625 					 0.7625 
Iteration 33 	 0.7625 					 0.7625 
Iteration 34 	 0.7625 					 0.7625 
Iteration 35 	 0.7625 					 0.7625 
Iteration 36 	 0.7625 					 0

Iteration 32 	 0.76875 					 0.79375 
Iteration 33 	 0.76875 					 0.79375 
Iteration 34 	 0.76875 					 0.79375 
Iteration 35 	 0.76875 					 0.79375 
Iteration 36 	 0.76875 					 0.79375 
Iteration 37 	 0.76875 					 0.79375 
Iteration 38 	 0.76875 					 0.79375 
Iteration 39 	 0.76875 					 0.79375 
Iteration 40 	 0.76875 					 0.79375 
Iteration 41 	 0.76875 					 0.79375 
Iteration 42 	 0.76875 					 0.79375 
Iteration 43 	 0.76875 					 0.79375 
Iteration 44 	 0.76875 					 0.79375 
Iteration 45 	 0.76875 					 0.79375 
Iteration 46 	 0.76875 					 0.79375 
Iteration 47 	 0.76875 					 0.79375 
Iteration 48 	 0.76875 					 0.79375 
Iteration 49 	 0.76875 					 0.79375 


 ----------------------------------------------------------------------------------------
Run Number - 14
		 Best value of metric across iteration 	 Best value of metric across population  
Iteration 0 	 0.75625 					 0.75625 
Iteration 1 	 0.75625 					 0.75625 
Iteration 2 	 0.74375 					 0.75625 
Iteration 3 	

		 Best value of metric across iteration 	 Best value of metric across population  
Iteration 0 	 0.7375 					 0.7375 
Iteration 1 	 0.74375 					 0.74375 
Iteration 2 	 0.75 					 0.75 
Iteration 3 	 0.75 					 0.75 
Iteration 4 	 0.75625 					 0.75625 
Iteration 5 	 0.75 					 0.75625 
Iteration 6 	 0.7875 					 0.7875 
Iteration 7 	 0.76875 					 0.7875 
Iteration 8 	 0.76875 					 0.7875 
Iteration 9 	 0.76875 					 0.7875 
Iteration 10 	 0.76875 					 0.7875 
Iteration 11 	 0.76875 					 0.7875 
Iteration 12 	 0.76875 					 0.7875 
Iteration 13 	 0.76875 					 0.7875 
Iteration 14 	 0.76875 					 0.7875 
Iteration 15 	 0.76875 					 0.7875 
Iteration 16 	 0.76875 					 0.7875 
Iteration 17 	 0.76875 					 0.7875 
Iteration 18 	 0.76875 					 0.7875 
Iteration 19 	 0.76875 					 0.7875 
Iteration 20 	 0.76875 					 0.7875 
Iteration 21 	 0.76875 					 0.7875 
Iteration 22 	 0.76875 					 0.7875 
Iteration 23 	 0.76875 					 0.7875 
Iteration 24 	 0.76875 					 0.7875 
Iteration 25 

run_idx  objective_scores  best_scores  num_features  \
0         1           0.75625      0.75625           249   
1         2           0.76250      0.76250           253   
2         3           0.77500      0.77500           254   
3         4           0.76250      0.76250           267   
4         5           0.76250      0.76250           258   
5         6           0.76875      0.76875           246   
6         7           0.77500      0.77500           253   
7         8           0.77500      0.77500           244   
8         9           0.76250      0.76250           233   
9        10           0.76875      0.76875           254   
10       11           0.76250      0.76250           263   
11       12           0.78125      0.78125           259   
12       13           0.79375      0.79375           249   
13       14           0.76250      0.76250           268   
14       15           0.79375      0.79375           261   
15       16           0.77500      0.77500           262   
16       17           0.78750      0.78750           264   
17       18           0.78750      0.78750           281   
18       19           0.78125      0.78125           257   
19       20           0.77500      0.77500           246   

                                    selected_features  \
0   [AMR, apol, naAromAtom, nN, nO, ATS0m, ATS1m, ...   
1   [ALogP, naAromAtom, nAromBond, nHeavyAtom, nX,...   
2   [ALogp2, apol, nAromBond, nC, nN, nO, ATS3m, A...   
3   [ALogP, AMR, apol, naAromAtom, nAromBond, nAto...   
4   [ALogP, ALogp2, naAromAtom, nAromBond, nHeavyA...   
5   [ALogp2, AMR, apol, naAromAtom, nAromBond, nHe...   
6   [ALogP, apol, naAromAtom, nHeavyAtom, nH, nC, ...   
7   [AMR, nHeavyAtom, nO, nF, ATS1m, ATS4m, ATS5m,...   
8   [ALogp2, naAromAtom, nH, nC, nN, nO, ATS0m, AT...   
9   [ALogP, apol, naAromAtom, nHeavyAtom, nC, nN, ...   
10  [naAromAtom, nAromBond, nHeavyAtom, nN, nF, nX...   
11  [ALogP, apol, naAromAtom, nAromBond, nHeavyAto...   
12  [ALogP, ALogp2, AMR, nAtom, nN, nO, nF, ATS0m,...   
13  [ALogP, apol, naAromAtom, nAtom, nC, ATS0m, AT...   
14  [naAromAtom, nAromBond, nAtom, nHeavyAtom, nH,...   
15  [ALogP, AMR, nAromBond, nC, nN, nO, nF, ATS0m,...   
16  [AMR, apol, naAromAtom, nAtom, nH, nC, nO, nF,...   
17  [ALogP, ALogp2, AMR, apol, nHeavyAtom, nH, nN,...   
18  [nAromBond, nAtom, nC, nF, ATS0m, ATS3m, ATS4m...   
19  [apol, nN, nX, ATS1m, ATS3m, ATS2v, ATS5v, ATS...   

                                                 plot  
0   Figure({\n    'data': [{'mode': 'markers',\n  ...  
1   Figure({\n    'data': [{'mode': 'markers',\n  ...  
2   Figure({\n    'data': [{'mode': 'markers',\n  ...  
3   Figure({\n    'data': [{'mode': 'markers',\n  ...  
4   Figure({\n    'data': [{'mode': 'markers',\n  ...  
5   Figure({\n    'data': [{'mode': 'markers',\n  ...  
6   Figure({\n    'data': [{'mode': 'markers',\n  ...  
7   Figure({\n    'data': [{'mode': 'markers',\n  ...  
8   Figure({\n    'data': [{'mode': 'markers',\n  ...  
9   Figure({\n    'data': [{'mode': 'markers',\n  ...  
10  Figure({\n    'data': [{'mode': 'markers',\n  ...  
11  Figure({\n    'data': [{'mode': 'markers',\n  ...  
12  Figure({\n    'data': [{'mode': 'markers',\n  ...  
13  Figure({\n    'data': [{'mode': 'markers',\n  ...  
14  Figure({\n    'data': [{'mode': 'markers',\n  ...  
15  Figure({\n    'data': [{'mode': 'markers',\n  ...  
16  Figure({\n    'data': [{'mode': 'markers',\n  ...  
17  Figure({\n    'data': [{'mode': 'markers',\n  ...  
18  Figure({\n    'data': [{'mode': 'markers',\n  ...  
19  Figure({\n    'data': [{'mode': 'markers',\n  ...

# Gradient Boosting

In [22]:
start = time.time()

gradient_boost_df = multiple_runtime(20, gbc_model, X_train, y_train, X_valid, y_valid)

end = time.time()
print("time =" ,((end - start)%60), " Menit")
print("\n\n ----------------------------------------------------------------------------------------")
gradient_boost_df



 ----------------------------------------------------------------------------------------
Run Number - 1
		 Best value of metric across iteration 	 Best value of metric across population  
Iteration 0 	 0.76875 					 0.76875 
Iteration 1 	 0.7625 					 0.76875 
Iteration 2 	 0.75 					 0.76875 
Iteration 3 	 0.75 					 0.76875 
Iteration 4 	 0.74375 					 0.76875 
Iteration 5 	 0.74375 					 0.76875 
Iteration 6 	 0.75625 					 0.76875 
Iteration 7 	 0.75625 					 0.76875 
Iteration 8 	 0.75 					 0.76875 
Iteration 9 	 0.75 					 0.76875 
Iteration 10 	 0.75 					 0.76875 
Iteration 11 	 0.75 					 0.76875 
Iteration 12 	 0.75 					 0.76875 
Iteration 13 	 0.75 					 0.76875 
Iteration 14 	 0.75 					 0.76875 
Iteration 15 	 0.75 					 0.76875 
Iteration 16 	 0.75 					 0.76875 
Iteration 17 	 0.75 					 0.76875 
Iteration 18 	 0.75 					 0.76875 
Iteration 19 	 0.75 					 0.76875 
Iteration 20 	 0.75 					 0.76875 
Iteration 21 	 0.75 					 0.76875 
Iteration 22 	 0.75 					 0.7687

Iteration 1 	 0.76875 					 0.76875 
Iteration 2 	 0.75625 					 0.76875 
Iteration 3 	 0.75625 					 0.76875 
Iteration 4 	 0.75625 					 0.76875 
Iteration 5 	 0.75625 					 0.76875 
Iteration 6 	 0.75625 					 0.76875 
Iteration 7 	 0.75625 					 0.76875 
Iteration 8 	 0.75625 					 0.76875 
Iteration 9 	 0.75625 					 0.76875 
Iteration 10 	 0.75625 					 0.76875 
Iteration 11 	 0.75625 					 0.76875 
Iteration 12 	 0.75625 					 0.76875 
Iteration 13 	 0.75625 					 0.76875 
Iteration 14 	 0.75625 					 0.76875 
Iteration 15 	 0.75625 					 0.76875 
Iteration 16 	 0.75625 					 0.76875 
Iteration 17 	 0.75625 					 0.76875 
Iteration 18 	 0.75625 					 0.76875 
Iteration 19 	 0.75625 					 0.76875 
Iteration 20 	 0.75625 					 0.76875 
Iteration 21 	 0.75625 					 0.76875 
Iteration 22 	 0.75625 					 0.76875 
Iteration 23 	 0.75625 					 0.76875 
Iteration 24 	 0.75625 					 0.76875 
Iteration 25 	 0.75625 					 0.76875 
Iteration 26 	 0.75625 					 0.76875 
Iteration 27 	 0.7562

Iteration 1 	 0.7875 					 0.7875 
Iteration 2 	 0.78125 					 0.7875 
Iteration 3 	 0.78125 					 0.7875 
Iteration 4 	 0.78125 					 0.7875 
Iteration 5 	 0.78125 					 0.7875 
Iteration 6 	 0.78125 					 0.7875 
Iteration 7 	 0.78125 					 0.7875 
Iteration 8 	 0.78125 					 0.7875 
Iteration 9 	 0.78125 					 0.7875 
Iteration 10 	 0.78125 					 0.7875 
Iteration 11 	 0.78125 					 0.7875 
Iteration 12 	 0.78125 					 0.7875 
Iteration 13 	 0.78125 					 0.7875 
Iteration 14 	 0.78125 					 0.7875 
Iteration 15 	 0.78125 					 0.7875 
Iteration 16 	 0.78125 					 0.7875 
Iteration 17 	 0.78125 					 0.7875 
Iteration 18 	 0.78125 					 0.7875 
Iteration 19 	 0.78125 					 0.7875 
Iteration 20 	 0.78125 					 0.7875 
Iteration 21 	 0.78125 					 0.7875 
Iteration 22 	 0.78125 					 0.7875 
Iteration 23 	 0.78125 					 0.7875 
Iteration 24 	 0.78125 					 0.7875 
Iteration 25 	 0.78125 					 0.7875 
Iteration 26 	 0.78125 					 0.7875 
Iteration 27 	 0.78125 					 0.7875 
Iteration 2

Iteration 6 	 0.76875 					 0.76875 
Iteration 7 	 0.76875 					 0.76875 
Iteration 8 	 0.76875 					 0.76875 
Iteration 9 	 0.76875 					 0.76875 
Iteration 10 	 0.76875 					 0.76875 
Iteration 11 	 0.76875 					 0.76875 
Iteration 12 	 0.76875 					 0.76875 
Iteration 13 	 0.76875 					 0.76875 
Iteration 14 	 0.76875 					 0.76875 
Iteration 15 	 0.76875 					 0.76875 
Iteration 16 	 0.76875 					 0.76875 
Iteration 17 	 0.76875 					 0.76875 
Iteration 18 	 0.76875 					 0.76875 
Iteration 19 	 0.76875 					 0.76875 
Iteration 20 	 0.76875 					 0.76875 
Iteration 21 	 0.76875 					 0.76875 
Iteration 22 	 0.76875 					 0.76875 
Iteration 23 	 0.76875 					 0.76875 
Iteration 24 	 0.76875 					 0.76875 
Iteration 25 	 0.76875 					 0.76875 
Iteration 26 	 0.76875 					 0.76875 
Iteration 27 	 0.76875 					 0.76875 
Iteration 28 	 0.76875 					 0.76875 
Iteration 29 	 0.76875 					 0.76875 
Iteration 30 	 0.76875 					 0.76875 
Iteration 31 	 0.76875 					 0.76875 
Iteration 32 	 0

Iteration 9 	 0.775 					 0.7875 
Iteration 10 	 0.775 					 0.7875 
Iteration 11 	 0.775 					 0.7875 
Iteration 12 	 0.78125 					 0.7875 
Iteration 13 	 0.775 					 0.7875 
Iteration 14 	 0.78125 					 0.7875 
Iteration 15 	 0.7875 					 0.7875 
Iteration 16 	 0.78125 					 0.7875 
Iteration 17 	 0.78125 					 0.7875 
Iteration 18 	 0.78125 					 0.7875 
Iteration 19 	 0.78125 					 0.7875 
Iteration 20 	 0.78125 					 0.7875 
Iteration 21 	 0.78125 					 0.7875 
Iteration 22 	 0.78125 					 0.7875 
Iteration 23 	 0.78125 					 0.7875 
Iteration 24 	 0.78125 					 0.7875 
Iteration 25 	 0.78125 					 0.7875 
Iteration 26 	 0.78125 					 0.7875 
Iteration 27 	 0.78125 					 0.7875 
Iteration 28 	 0.78125 					 0.7875 
Iteration 29 	 0.78125 					 0.7875 
Iteration 30 	 0.78125 					 0.7875 
Iteration 31 	 0.78125 					 0.7875 
Iteration 32 	 0.78125 					 0.7875 
Iteration 33 	 0.78125 					 0.7875 
Iteration 34 	 0.78125 					 0.7875 
Iteration 35 	 0.78125 					 0.7875 
Iteration 3

run_idx  objective_scores  best_scores  num_features  \
0         1           0.76875      0.76875           254   
1         2           0.78125      0.78125           257   
2         3           0.76875      0.76875           260   
3         4           0.78125      0.78125           262   
4         5           0.76875      0.76875           250   
5         6           0.78125      0.78125           237   
6         7           0.79375      0.79375           255   
7         8           0.76875      0.76875           265   
8         9           0.78750      0.78750           252   
9        10           0.77500      0.77500           258   
10       11           0.76875      0.76875           258   
11       12           0.77500      0.77500           240   
12       13           0.76875      0.76875           257   
13       14           0.78125      0.78125           245   
14       15           0.77500      0.77500           247   
15       16           0.76875      0.76875           256   
16       17           0.78750      0.78750           238   
17       18           0.78750      0.78750           251   
18       19           0.76250      0.76250           271   
19       20           0.77500      0.77500           247   

                                    selected_features  \
0   [apol, naAromAtom, nAtom, nH, nC, nN, nF, nX, ...   
1   [ALogP, nAromBond, nC, nO, nF, ATS0m, ATS2m, A...   
2   [ALogP, apol, nAtom, nH, nN, nO, nF, ATS1m, AT...   
3   [apol, nAromBond, nH, nC, nN, nF, nX, ATS4m, A...   
4   [ALogP, ALogp2, naAromAtom, nAromBond, nAtom, ...   
5   [ALogP, AMR, apol, naAromAtom, nAtom, nHeavyAt...   
6   [ALogp2, apol, nAromBond, nAtom, nO, nX, ATS2m...   
7   [AMR, naAromAtom, nAromBond, nAtom, nH, nN, nO...   
8   [nHeavyAtom, nH, nN, nO, ATS1m, ATS3m, ATS4m, ...   
9   [ALogP, ALogp2, AMR, naAromAtom, nH, nF, ATS0m...   
10  [ALogP, ALogp2, apol, nAromBond, nAtom, nH, nO...   
11  [ALogP, apol, nAromBond, nHeavyAtom, nN, ATS1m...   
12  [ALogp2, AMR, apol, nO, nF, ATS0m, ATS1m, ATS5...   
13  [ALogp2, apol, nAromBond, nAtom, nC, nN, nO, n...   
14  [naAromAtom, nAromBond, nAtom, nHeavyAtom, nC,...   
15  [ALogp2, AMR, naAromAtom, nAromBond, nAtom, nH...   
16  [ALogp2, AMR, apol, naAromAtom, nC, nF, nX, AT...   
17  [ALogP, apol, naAromAtom, nAromBond, nH, nF, n...   
18  [ALogP, ALogp2, AMR, naAromAtom, nAtom, nHeavy...   
19  [ALogp2, AMR, apol, naAromAtom, ATS1m, ATS3m, ...   

                                                 plot  
0   Figure({\n    'data': [{'mode': 'markers',\n  ...  
1   Figure({\n    'data': [{'mode': 'markers',\n  ...  
2   Figure({\n    'data': [{'mode': 'markers',\n  ...  
3   Figure({\n    'data': [{'mode': 'markers',\n  ...  
4   Figure({\n    'data': [{'mode': 'markers',\n  ...  
5   Figure({\n    'data': [{'mode': 'markers',\n  ...  
6   Figure({\n    'data': [{'mode': 'markers',\n  ...  
7   Figure({\n    'data': [{'mode': 'markers',\n  ...  
8   Figure({\n    'data': [{'mode': 'markers',\n  ...  
9   Figure({\n    'data': [{'mode': 'markers',\n  ...  
10  Figure({\n    'data': [{'mode': 'markers',\n  ...  
11  Figure({\n    'data': [{'mode': 'markers',\n  ...  
12  Figure({\n    'data': [{'mode': 'markers',\n  ...  
13  Figure({\n    'data': [{'mode': 'markers',\n  ...  
14  Figure({\n    'data': [{'mode': 'markers',\n  ...  
15  Figure({\n    'data': [{'mode': 'markers',\n  ...  
16  Figure({\n    'data': [{'mode': 'markers',\n  ...  
17  Figure({\n    'data': [{'mode': 'markers',\n  ...  
18  Figure({\n    'data': [{'mode': 'markers',\n  ...  
19  Figure({\n    'data': [{'mode': 'markers',\n  ...

In [23]:
random_forest_df.to_pickle('/Users/irfanularifa/Documents/TA_FIX/0. Dataset/GA-RFC.pickle')
ada_boost_df.to_pickle('/Users/irfanularifa/Documents/TA_FIX/0. Dataset/GA-ABC.pickle')
gradient_boost_df.to_pickle('/Users/irfanularifa/Documents/TA_FIX/0. Dataset/GA-GBC.pickle')